ideia:

- ao clicar na bola, site com estat sobre a palavra?

- selecionar threshold dos 0.05 com barrinha mesmo no html?

- mudar cor para verde, neutro, vermelho: pq o neutro parece verde, ent vamos meter tp amarelo ou assim idk, dps ve se

In [1]:
import pandas as pd

df = pd.read_parquet('data05.parquet')
df["keywords"] = df["keywords"].map(lambda dic: {key: dic[key] for key in dic.keys() if dic[key] != None and dic[key]["filter"] > 0.1})

In [4]:
import networkx as nx
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np


# Sample dictionary data
linha = 0
data = df["keywords"].iloc[linha]
company = df.index[linha]

# Create the graph
G = nx.Graph()

# Add nodes to the graph with attributes
for word, attributes in data.items():
    G.add_node(word, **attributes)
    G.add_edge(company, word)

# Node positions
pos = nx.spring_layout(G, seed=42)

# Lists for node positions and info
node_x = []
node_y = []
node_color = []
node_text = []  # Label to show only the word
node_hovertext = []  # Detailed info for hover

for node in G.nodes:
    if node == company:
        x, y = 0, 0
        node_x.append(x)
        node_y.append(y)
        node_text.append(node)
        node_hovertext.append(f"Company: {node}")
        node_color.append("black")
        continue
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    
    # Color the node from red (-1) to green (1)
    color_value = (G.nodes[node]["sentiment"] + 1) / 2  # Normalize between 0 and 1
    node_color.append(f"rgb({255 * (1 - color_value)}, {255 * color_value}, 0)")
    
    # Text for label (only the word) and hover (detailed info)
    node_text.append(node)
    node_hovertext.append(
        f"""Palavra: {node}
        <br>Contagem: {G.nodes[node]['count']}
        <br>Sentiment: {round(G.nodes[node]['sentiment'], 3)}
        <br>Source:
        <br>{G.nodes[node]['source']}"""
        #<br>News: {random.choice(list(G.nodes[node]['news']))} #arranjar melhor forma de fazer isto
    )

# Create the plotly figure
fig = make_subplots()

# Draw edges
edge_x = []
edge_y = []

for edge in G.edges:
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

fig.add_trace(
    go.Scatter(
        x=edge_x,
        y=edge_y,
        line=dict(width=1, color="gray"),
        hoverinfo="none",
        mode="lines"
    )
)

# Draw nodes
fig.add_trace(
    go.Scatter(
        x=node_x,
        y=node_y,
        mode="markers+text",
        text=node_text,  # Shows only the word as label
        hovertext=node_hovertext,  # Detailed info on hover
        marker=dict(
            color=node_color,
            size=20,
            line=dict(color="black", width=1)
        ),
        hoverinfo="text"  # Use hovertext for detailed info
    )
)

# Figure layout
fig.update_layout(
    showlegend=False,
    hovermode="closest",
    margin=dict(b=0, l=0, r=0, t=0),
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
)

fig.write_html("grafo_interativo.html")
fig.show()


In [3]:
#import webbrowser
#webbrowser.open("grafo_interativo.html")

In [64]:
import networkx as nx
import plotly.graph_objects as go
import pandas as pd

# Sample data (You should replace this with your actual dataframe)
df = pd.read_parquet('data05.parquet')
df["keywords"] = df["keywords"].map(lambda dic: {key: dic[key] for key in dic.keys() if dic[key] is not None and dic[key]["filter"] > 0.1})

# Using the first row as an example
linha = 0
data = df["keywords"].iloc[linha]
company = "Company A"

# Create the graph
G = nx.Graph()

# Add nodes to the graph with attributes
for word, attributes in data.items():
    G.add_node(word, **attributes)
    G.add_edge(company, word)

# Node positions
pos = nx.spring_layout(G, seed=42)

# Lists for node positions and info
node_x = []
node_y = []
node_color = []
node_text = []
node_hovertext = []

for node in G.nodes:
    if node == company:
        x, y = 0, 0
        node_x.append(x)
        node_y.append(y)
        node_text.append(node)
        node_hovertext.append(f"Company: {node}")
        node_color.append("black")
        continue
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    
    # Color from red (-1) to green (1)
    color_value = (G.nodes[node]["sentiment"] + 1) / 2
    node_color.append(f"rgb({255 * (1 - color_value)}, {255 * color_value}, 0)")
    
    node_text.append(node)
    node_hovertext.append(
        f"""Word: {node}<br>Count: {G.nodes[node]['count']}<br>Sentiment: {round(G.nodes[node]['sentiment'], 3)}<br>Source: {G.nodes[node]['source']}"""
    )

# Create the Plotly figure
fig = go.Figure()

# Draw edges
edge_x = []
edge_y = []

for edge in G.edges:
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

fig.add_trace(
    go.Scatter(
        x=edge_x,
        y=edge_y,
        line=dict(width=1, color="gray"),
        hoverinfo="none",
        mode="lines"
    )
)

# Draw nodes
fig.add_trace(
    go.Scatter(
        x=node_x,
        y=node_y,
        mode="markers+text",
        text=node_text,
        hovertext=node_hovertext,
        marker=dict(
            color=node_color,
            size=20,
            line=dict(color="black", width=1)
        ),
        hoverinfo="text"
    )
)

fig.update_layout(
    showlegend=False,
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    margin=dict(l=0, r=0, t=0, b=0)  # Remove all margins
)

# Generate the HTML with full document structure
html_code = fig.to_html(include_plotlyjs='cdn', full_html=True)

# Modify the generated HTML to include the side panel and additional styles/scripts
additional_html = """
<style>
    body, html {
        height: 100%;
        margin: 0;
        font-family: Arial, sans-serif;
        overflow: hidden;
    }
    #graph {
        position: absolute;
        top: 0;
        left: 0;
        right: 0;
        bottom: 0;
        z-index: 1; /* Ensure the graph is behind the panel */
    }
    #info-panel {
        position: absolute;
        right: 0; /* Default to right */
        top: 0;
        width: 300px; /* Fixed width of the panel */
        height: 100vh; /* Full height of container */
        background-color: #f4f4f9;
        box-shadow: -2px 0 5px rgba(0,0,0,0.1);
        transform: translateX(100%); /* Hide initially */
        transition: transform 0.3s ease; /* Smooth slide in */
        z-index: 2; /* Panel above the graph */
        display: flex;
        flex-direction: column; /* Arrange children vertically */
    }
    #info-panel.open {
        transform: translateX(0); /* Slide in the panel */
    }
    .close-button {
        background-color: #ff4c4c;
        color: white;
        border: none;
        padding: 10px;
        cursor: pointer;
        float: right;
        margin-bottom: 10px;
    }

    #scrollable-content {
        flex: 1; /* Take remaining vertical space */
        max-height: 400px; /* Adjust height as needed */
        overflow-y: auto; /* Enable vertical scrolling */
        padding: 10px; /* Padding for inner content */
        background-color: #ffffff; /* Background color */
        border: 1px solid #ddd; /* Border for the scrollable area */
        box-shadow: inset 0 0 5px rgba(0,0,0,0.1); /* Inner shadow */
        margin-top: 10px; /* Spacing from the title */
        }
</style>

<div id="info-panel">
    <button class="close-button" onclick="closePanel()">Close</button>
    <h2>Node Information</h2>
    <p id="node-info">Click a node to view details</p>
    <div id="scrollable-content">
        <ul>
            <li><a href="http://example.com" target="_blank">Example URL 1</a></li>
            <li><a href="http://example.org" target="_blank">Example URL 2</a></li>
            <li><a href="http://example.net" target="_blank">Example URL 3</a></li>
            <li><a href="http://example.net" target="_blank">Example URL 3</a></li>
            <li><a href="http://example.net" target="_blank">Example URL 3</a></li>
            <li><a href="http://example.net" target="_blank">Example URL 3</a></li>
            <li><a href="http://example.net" target="_blank">Example URL 3</a></li>
            <li><a href="http://example.net" target="_blank">Example URL 3</a></li>
            <li><a href="http://example.net" target="_blank">Example URL 3</a></li>
            <li><a href="http://example.net" target="_blank">Example URL 3</a></li>
            <li><a href="http://example.net" target="_blank">Example URL 3</a></li>
            <li><a href="http://example.net" target="_blank">Example URL 3</a></li>
            <li><a href="http://example.net" target="_blank">Example URL 3</a></li>
            <li><a href="http://example.net" target="_blank">Example URL 3</a></li>
            <li><a href="http://example.net" target="_blank">Example URL 3</a></li>
            <li><a href="http://example.net" target="_blank">Example URL 3</a></li>
            <li><a href="http://example.net" target="_blank">Example URL 3</a></li>
            <li><a href="http://example.net" target="_blank">Example URL 3</a></li>
            <li><a href="http://example.net" target="_blank">Example URL 3</a></li>
            <li><a href="http://example.net" target="_blank">Example URL 3</a></li>
            <li><a href="http://example.net" target="_blank">Example URL 3</a></li>
            <li><a href="http://example.net" target="_blank">Example URL 3</a></li>
            <li><a href="http://example.net" target="_blank">Example URL 3</a></li>
            <li><a href="http://example.net" target="_blank">Example URL 3</a></li>
            <li><a href="http://example.net" target="_blank">Example URL 3</a></li>
            <li><a href="http://example.net" target="_blank">Example URL 3</a></li>
            <li><a href="http://example.net" target="_blank">Example URL 3</a></li>
            <li><a href="http://example.net" target="_blank">Example URL 3</a></li>
        </ul>
    </div>
    <p> iupi </p>
</div>

<script>
    function closePanel() {
        document.getElementById('info-panel').classList.remove('open');
    }

    document.addEventListener('DOMContentLoaded', function() {
        var plotDiv = document.querySelector('.plotly-graph-div');
        
        if (plotDiv) {
            plotDiv.on('plotly_click', function(data) {
                if (data.points.length > 0) {
                    var point = data.points[0];
                    document.getElementById('node-info').innerHTML = point.hovertext;

                    // Determine mouse click position for panel placement
                    var mouseX = data.event.clientX; // Get mouse X position
                    var panel = document.getElementById('info-panel');

                    // Adjust panel position based on click location
                    if (mouseX > window.innerWidth / 2) {
                        panel.style.right = 'auto'; // Clear right positioning
                        panel.style.left = '0'; // Show on the left side
                    } else {
                        panel.style.left = 'auto'; // Clear left positioning
                        panel.style.right = '0'; // Show on the right side
                    }

                    // Show the panel
                    panel.classList.add('open'); // Show panel on click
                }
            });
        } else {
            console.error("Graph div not found.");
        }
    });
</script>
"""

# Combine the Plotly HTML and the additional HTML
html_code = html_code.replace("</body>", additional_html + "</body>")

# Save to HTML file
with open("interactive_graph.html", "w") as file:
    file.write(html_code)

print("HTML file with interactive graph and side panel created as 'interactive_graph.html'.")


HTML file with interactive graph and side panel created as 'interactive_graph.html'.


---
---
---
---
---
---
---
---
---
---
---